In [1]:
pip install apyori

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path 

# association rule mining 
from mlxtend.frequent_patterns import apriori, association_rules
#encoder 
from mlxtend.preprocessing import TransactionEncoder

from apyori import apriori

In [3]:
df = pd.read_csv('/Users/sudeshgurung/Msc-Data_Analytics/ADTA_with_python/Course_work_1/Census.csv');
df.sample(10)



,Person ID,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,No of hours,Approximated Social Grade
516538,7900938,E12000007,H,2,1,2,3,1,2,1,2,3,2,1,2,10,3,42.0,1
230713,7838716,E12000003,H,2,1,2,4,2,2,1,2,1,2,3,6,11,-9,NaN,3
463953,7708016,E12000008,H,1,1,1,3,1,2,1,2,1,1,2,8,3,3,47.0,3
269807,7915750,E12000007,H,1,1,2,7,4,2,1,1,1,3,5,3,6,-9,NaN,2
71406,7534440,E12000004,H,2,1,2,8,2,2,1,3,1,2,5,8,2,-9,NaN,4
436108,7601520,E12000006,H,1,1,1,3,1,2,2,2,2,1,1,7,4,3,38.0,4
20966,7437638,E12000009,H,3,1,2,1,1,1,1,1,1,2,-9,-9,-9,-9,NaN,-9
384988,7411729,E12000003,H,3,1,1,3,1,2,1,2,1,2,1,2,2,3,46.0,2
96839,7586117,E12000008,H,2,1,2,7,2,2,1,2,1,1,5,6,6,-9,NaN,3
309396,7547248,E12000009,H,2,1,2,2,1,1,1,2,1,2,4,9,5,1,9.0,2


In [4]:
#removing -9 from all the columns 

# df.drop(df[df['Family Composition'] == -9].index, inplace = True)
# df.drop(df[df['Country of Birth']== -9].index, inplace = True)
# df.drop(df[df['Health']== -9].index, inplace = True)
# df.drop(df[df['Ethnic Group']== -9].index, inplace = True)
# df.drop(df[df['Religion']== -9].index, inplace = True)
# df.drop(df[df['Economic Activity']== -9].index, inplace = True)
# df.drop(df[df['Occupation']== -9].index, inplace = True)
# df.drop(df[df['Industry']== -9].index, inplace = True)
# df.drop(df[df['Hours worked per week']== -9].index, inplace = True)
# df.drop(df[df['Approximated Social Grade']== -9].index, inplace = True)
# df

In [5]:
df.isna().sum()

Person ID                         0
Region                            0
Residence Type                    0
Family Composition                0
Population Base                   0
Sex                               0
Age                               0
Marital Status                    0
Student                           0
Country of Birth                  0
Health                            0
Ethnic Group                      0
Religion                          0
Economic Activity                 0
Occupation                        0
Industry                          0
Hours worked per week             0
No of hours                  302321
Approximated Social Grade         0
dtype: int64

In [6]:
#mapping 
family_comp = {
    1 : "Not in a family", 
    2: "Married/same-sex civil partnership couple family", 
    3: "Cohabiting couple family",
    4: "Lone parent family (male head)",
    5: "Lone parent family (female head)", 
    6: "Other related family", 
    -9: "Family Composition: No code required"
}
population_base = {
    1: "Usual resident",
    2: "Student living away from home during term-time",
    3: "Short-term resident"
}
sex = {
    1: "Male",
    2: "Female"
}
age = {
    1: "0to15",
    2: "16to24",
    3: "25to34",
    4:"35to44",
    5: "45to54",
    6: "55to64",
    7: "65to74",
    8: "75 and over"
}
marital_status = {
    1: "Single (never married or never registered a same-sex civil partnership)",
    2: "Married or in a registered same-sex civil partnership",
    3: "Separated but still legally married or separated but still legally in a same-sex civil partnership",
    4: "Divorced or formerly in a same-sex civil partnership which is now legally dissolved",
    5: "Widowed or surviving partner from a same-sex civil partnership",
}
student = {
    1:" Yes",
    2: "No"
}
country_of_birth = {
    1: "UK",
    2: "Non UK",
    -9: "Country of birth: No code required"
}
health = {
    1: "Very good health",
    2: "Good health",
    3: "Fair health",
    4: "Bad health",
    5: "Very bad health",
    -9: "Health: No code required"
}
ethnic_group = {
    1: "White", 
    2: "Mixed", 
    3: "Asian and Asian British", 
    4: "Black or Black British", 
    5: "Chinese or Other ethnic group",
    -9: "Ethnic Group: No code required"
}
religion = {
    1: "No religion",
    2: "Christian",
    3: "Buddhist",
    4: "Hindu",
    5: "Jewish",
    6: "Muslim",
    7: "Sikh",
    8: "Other religion",
    9: "Not stated",
    -9:"Religion: No code required "
}
economic_activity = {
    1: "Economically active: Employee",
    2: "Economically active: Self-employed",
    3: "Economically active: Unemployed",
    4: "Economically active: Full-time student",
    5: "Economically inactive: Retired",
    6: "Economically inactive: Student",
    7:" Economically inactive: Looking after home or family",
    8: "Economically inactive: Long-term sick or disabled",
    9: "Economically inactive: Other",
    -9: "Economic Activity: No code required",
}
occupation = {
    1: "Managers, Directors and Senior Officials",
    2: "Professional Occupations",
    3:" Associate Professional and Technical Occupations",
    4: "Administrative and Secretarial Occupations",
    5: "Skilled Trades Occupations",
    6:" Caring, Leisure and Other Service Occupations",
    7: "Sales and Customer Service Occupations",
    8: "Process, Plant and Machine Operatives",
    9: "Elementary Occupations",
    -9: "Occupation: No code required ",
    }
industry = {
    1: "Agriculture, forestry and fishing",
    2: "Mining and quarrying; Manufacturing;Electricity, gas, steam and air conditioning system; Water supply",
    3: "Construction",
    4: "Wholesale and retail trade; Repair of motor vehicles and motorcycles",
    5: "Accommodation and food service activities",
    6: "Transport and storage; Information and communication",
    7: "Financial and insurance activities; Intermediation",
    8: "Real estate activities; Professional, scientific and technical activities; Administrative and support service activities",
    9: "Public administration and defence; compulsory social security",
    10: "Education",
    11: "Human health and social work activities",
    12: "Other community, social and personal service activities; Private households employing domestic staff; Extra-territorial organisations and bodies",
    -9: "Industry: No code required "
}
hours_worked_per_week = {
    1: "Part-time: 15 or less hours worked",
    2:"Part-time: 16 to 30 hours worked",
    3: "Full-time: 31 to 48 hours worked",
    4: "Full-time: 49 or more hours worked",
    -9: "Hours worked per week: No code required",
}
approx_sc_grade = {
    1: "AB",
    2: "C1",
    3: "C2",
    4: "DE",
    -9: "Approximated social grade: No code required"
}

In [7]:
df['Family Composition'].replace(family_comp, inplace = True)
df['Population Base'].replace(population_base, inplace = True)
df['Sex'].replace(sex, inplace = True)
df['Age'].replace(age, inplace = True)
df['Marital Status'].replace(marital_status, inplace = True)
df['Student'].replace(student, inplace = True)
df['Country of Birth'].replace(country_of_birth, inplace = True)
df['Health'].replace(health, inplace = True)
df['Ethnic Group'].replace(ethnic_group, inplace = True)
df['Religion'].replace(religion, inplace = True)
df['Economic Activity'].replace(economic_activity, inplace = True)
df['Occupation'].replace(occupation, inplace = True)
df['Industry'].replace(industry, inplace = True)
df['Hours worked per week'].replace(hours_worked_per_week, inplace = True)
df['Approximated Social Grade'].replace(approx_sc_grade, inplace = True)
df.head()

,Person ID,Region,Residence Type,Family Composition,Population Base,Sex,Age,Marital Status,Student,Country of Birth,Health,Ethnic Group,Religion,Economic Activity,Occupation,Industry,Hours worked per week,No of hours,Approximated Social Grade
0,7394816,E12000001,H,Married/same-sex civil partnership couple family,Usual resident,Female,55to64,Married or in a registered same-sex civil part...,No,UK,Good health,White,Christian,Economically inactive: Retired,"Process, Plant and Machine Operatives",Mining and quarrying; Manufacturing;Electricit...,Hours worked per week: No code required,NaN,DE
1,7394832,E12000001,H,Cohabiting couple family,Usual resident,Female,0to15,Single (never married or never registered a sa...,No,UK,Good health,White,No religion,Economic Activity: No code required,Occupation: No code required,Industry: No code required,Hours worked per week: No code required,NaN,Approximated social grade: No code required
2,7394719,E12000001,H,Married/same-sex civil partnership couple family,Usual resident,Male,65to74,Married or in a registered same-sex civil part...,No,UK,Very good health,White,Christian,Economically inactive: Retired,"Process, Plant and Machine Operatives",Mining and quarrying; Manufacturing;Electricit...,Hours worked per week: No code required,NaN,C1
3,7394840,E12000001,H,Not in a family,Usual resident,Female,55to64,Divorced or formerly in a same-sex civil partn...,No,UK,Fair health,White,Christian,Economically inactive: Retired,Elementary Occupations,Accommodation and food service activities,Hours worked per week: No code required,NaN,DE
4,7394711,E12000001,H,Married/same-sex civil partnership couple family,Usual resident,Male,0to15,Single (never married or never registered a sa...,Yes,UK,Very good health,White,No religion,Economic Activity: No code required,Occupation: No code required,Industry: No code required,Hours worked per week: No code required,NaN,Approximated social grade: No code required


In [8]:
# df = df.dropna()
# df = df.sample(frac=0.5)
# df = df.drop(columns= ['Person ID'])
df = df.drop(columns= ['No of hours', 'Person ID'])
df.shape

(569740, 17)

In [9]:
data_list = []
for row in range(0, 569740): 
    data_list.append([str(df.values[row,column]) for column in range(0, 17)])

data_list

[['E12000001',
  'H',
  'Married/same-sex civil partnership couple family',
  'Usual resident',
  'Female',
  '55to64',
  'Married or in a registered same-sex civil partnership',
  'No',
  'UK',
  'Good health',
  'White',
  'Christian',
  'Economically inactive: Retired',
  'Process, Plant and Machine Operatives',
  'Mining and quarrying; Manufacturing;Electricity, gas, steam and air conditioning system; Water supply',
  'Hours worked per week: No code required',
  'DE'],
 ['E12000001',
  'H',
  'Cohabiting couple family',
  'Usual resident',
  'Female',
  '0to15',
  'Single (never married or never registered a same-sex civil partnership)',
  'No',
  'UK',
  'Good health',
  'White',
  'No religion',
  'Economic Activity: No code required',
  'Occupation: No code required ',
  'Industry: No code required ',
  'Hours worked per week: No code required',
  'Approximated social grade: No code required'],
 ['E12000001',
  'H',
  'Married/same-sex civil partnership couple family',
  'Usual 

## Using Min-sup: 20

In [10]:
# rules = apriori(data_list, min_support = 0.3, min_confidence = 0.5, use_colnames = True, verbose = 1)
# rules = apriori(data_list, min_support = 0.4, min_confidence = 0.5, use_colnames = True, verbose = 1)
# rules = apriori(data_list, min_support = 0.45, min_confidence = 0.5, use_colnames = True, verbose = 1)
rules_20 = apriori(data_list, min_support = 0.2, min_confidence = 0.5, use_colnames = True, verbose = 1)

rules_20 

<generator object apriori at 0x7fd140f3dd60>

In [11]:
list_rules_20 = list(rules_20)
list_rules_20

[RelationRecord(items=frozenset({'Christian'}), support=0.5853213746621266, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Christian'}), confidence=0.5853213746621266, lift=1.0)]),
 RelationRecord(items=frozenset({'Female'}), support=0.5075508126513848, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Female'}), confidence=0.5075508126513848, lift=1.0)]),
 RelationRecord(items=frozenset({'H'}), support=0.9813002422157475, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'H'}), confidence=0.9813002422157475, lift=1.0)]),
 RelationRecord(items=frozenset({'Hours worked per week: No code required'}), support=0.5306297609435883, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Hours worked per week: No code required'}), confidence=0.5306297609435883, lift=1.0)]),
 RelationRecord(items=frozenset({'Married/same-sex civil partnership couple family'}), support

In [12]:
#Put the results into a dataframe to have the rules listed in a table
rules_df = pd.DataFrame(columns=['Items','Antecedent','Consequent','Support','Confidence','Lift'])
support = []
confidence = []
lift = []
antecedent = []
consequent = []
items = []

for RelationRecord in list_rules_20: 
    for ordered_stat in RelationRecord.ordered_statistics: 
        support.append(RelationRecord.support)
        confidence.append(ordered_stat.confidence)
        lift.append(ordered_stat.lift)
        items.append(RelationRecord.items)
        antecedent.append(ordered_stat.items_base)
        consequent.append(ordered_stat.items_add)

rules_df['Items'] = list(map(set, items))
rules_df['Antecedent'] = list(map(set, antecedent))
rules_df['Consequent'] = list(map(set, consequent))
rules_df['Support'] = support
rules_df['Confidence'] = confidence
rules_df['Lift'] = lift

# sorting the dataframe by lift
rules_df.sort_values(by='Lift', ascending = False, inplace = True)

In [15]:
above_one = rules_df[rules_df['Lift'] > 1]
above_one

,Items,Antecedent,Consequent,Support,Confidence,Lift
10021,"{Hours worked per week: No code required, H, U...","{Hours worked per week: No code required, H, U...","{Usual resident, Single (never married or neve...",0.207001,0.959313,4.187358
8732,"{H, Usual resident, Occupation: No code requir...",{Single (never married or never registered a s...,"{Occupation: No code required , H, UK}",0.207001,0.903552,4.187358
10042,"{Hours worked per week: No code required, H, U...","{Usual resident, Hours worked per week: No cod...","{H, UK, Industry: No code required }",0.207001,0.903552,4.187358
8725,"{H, Usual resident, Occupation: No code requir...","{H, UK, Occupation: No code required }","{Usual resident, Single (never married or neve...",0.207001,0.959313,4.187358
8735,"{H, Usual resident, Occupation: No code requir...","{Usual resident, Single (never married or neve...","{H, UK, Industry: No code required }",0.207001,0.903552,4.187358
...,...,...,...,...,...,...
6022,"{H, Usual resident, Male, White, UK}",{Male},"{H, Usual resident, UK, White}",0.380758,0.773192,1.000112
2737,"{H, Male, UK, White}",{Male},"{H, UK, White}",0.380758,0.773192,1.000112
61,"{Female, H}",{Female},{H},0.498076,0.981333,1.000033
62,"{Female, H}",{H},{Female},0.498076,0.507568,1.000033


In [18]:
#writing to a file
filepath = Path("/Users/sudeshgurung/Msc-Data_Analytics/ADTA_with_python/Course_work_1/rules_ARM/rules_20sup_50con.csv")
filepath.parent.mkdir(parents=True, exist_ok=True)  
above_one.to_csv(filepath) 


## Using min-sup: 50

In [19]:
rules_50 = apriori(data_list, min_support = 0.5, min_confidence = 0.5, use_colnames = True, verbose = 1)
rules_50

<generator object apriori at 0x7fd131a25040>

In [20]:
list_rules_50 = list(rules_50)
list_rules_50

[RelationRecord(items=frozenset({'Christian'}), support=0.5853213746621266, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Christian'}), confidence=0.5853213746621266, lift=1.0)]),
 RelationRecord(items=frozenset({'Female'}), support=0.5075508126513848, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Female'}), confidence=0.5075508126513848, lift=1.0)]),
 RelationRecord(items=frozenset({'H'}), support=0.9813002422157475, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'H'}), confidence=0.9813002422157475, lift=1.0)]),
 RelationRecord(items=frozenset({'Hours worked per week: No code required'}), support=0.5306297609435883, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Hours worked per week: No code required'}), confidence=0.5306297609435883, lift=1.0)]),
 RelationRecord(items=frozenset({'Married/same-sex civil partnership couple family'}), support

In [21]:
#Put the results into a dataframe to have the rules listed in a table
rules_df_50 = pd.DataFrame(columns=['Items','Antecedent','Consequent','Support','Confidence','Lift'])
support = []
confidence = []
lift = []
antecedent = []
consequent = []
items = []

for RelationRecord in list_rules_50: 
    for ordered_stat in RelationRecord.ordered_statistics: 
        support.append(RelationRecord.support)
        confidence.append(ordered_stat.confidence)
        lift.append(ordered_stat.lift)
        items.append(RelationRecord.items)
        antecedent.append(ordered_stat.items_base)
        consequent.append(ordered_stat.items_add)

rules_df_50['Items'] = list(map(set, items))
rules_df_50['Antecedent'] = list(map(set, antecedent))
rules_df_50['Consequent'] = list(map(set, consequent))
rules_df_50['Support'] = support
rules_df_50['Confidence'] = confidence
rules_df_50['Lift'] = lift

# sorting the dataframe by lift
rules_df_50.sort_values(by='Lift', ascending = False, inplace = True)

In [22]:
above_1 = rules_df_50[rules_df_50['Lift']>1]
above_1.head(20)

,Items,Antecedent,Consequent,Support,Confidence,Lift
224,"{Usual resident, Christian, UK, White}","{Christian, UK}","{Usual resident, White}",0.501109,0.961597,1.135295
229,"{Usual resident, Christian, UK, White}","{Usual resident, White}","{Christian, UK}",0.501109,0.591627,1.135295
223,"{Usual resident, Christian, UK, White}",{White},"{Usual resident, Christian, UK}",0.501109,0.590518,1.133167
94,"{Christian, UK, White}",{White},"{Christian, UK}",0.501109,0.590518,1.133167
230,"{Usual resident, Christian, UK, White}","{Usual resident, Christian, UK}",{White},0.501109,0.961597,1.133167
95,"{Christian, UK, White}","{Christian, UK}",{White},0.501109,0.961597,1.133167
319,"{H, Usual resident, White, No, UK}","{No, UK}","{H, Usual resident, White}",0.625473,0.936197,1.124964
330,"{H, Usual resident, White, No, UK}","{H, Usual resident, White}","{No, UK}",0.625473,0.751588,1.124964
256,"{H, No, UK, White}","{H, White}","{No, UK}",0.625473,0.750480,1.123305
318,"{H, Usual resident, White, No, UK}","{H, White}","{No, Usual resident, UK}",0.625473,0.750480,1.123305


In [23]:
#writing to a file
filepath = Path("/Users/sudeshgurung/Msc-Data_Analytics/ADTA_with_python/Course_work_1/rules_ARM/rules_dropna_50sup_50con.csv")
filepath.parent.mkdir(parents=True, exist_ok=True)  
above_1.to_csv(filepath)


## Using min-sup: 35

In [24]:
rules_35 = apriori(data_list, min_support = 0.35, min_confidence = 0.5, use_colnames = True, verbose = 1)
rules_35

<generator object apriori at 0x7fd131a25ba0>

In [25]:
list_rules_35 = list(rules_35)
list_rules_35

[RelationRecord(items=frozenset({'Christian'}), support=0.5853213746621266, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Christian'}), confidence=0.5853213746621266, lift=1.0)]),
 RelationRecord(items=frozenset({'Female'}), support=0.5075508126513848, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Female'}), confidence=0.5075508126513848, lift=1.0)]),
 RelationRecord(items=frozenset({'H'}), support=0.9813002422157475, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'H'}), confidence=0.9813002422157475, lift=1.0)]),
 RelationRecord(items=frozenset({'Hours worked per week: No code required'}), support=0.5306297609435883, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Hours worked per week: No code required'}), confidence=0.5306297609435883, lift=1.0)]),
 RelationRecord(items=frozenset({'Married/same-sex civil partnership couple family'}), support

In [26]:
#Put the results into a dataframe to have the rules listed in a table
rules_df_35 = pd.DataFrame(columns=['Items','Antecedent','Consequent','Support','Confidence','Lift'])
support = []
confidence = []
lift = []
antecedent = []
consequent = []
items = []

for RelationRecord in list_rules_35: 
    for ordered_stat in RelationRecord.ordered_statistics: 
        support.append(RelationRecord.support)
        confidence.append(ordered_stat.confidence)
        lift.append(ordered_stat.lift)
        items.append(RelationRecord.items)
        antecedent.append(ordered_stat.items_base)
        consequent.append(ordered_stat.items_add)

rules_df_35['Items'] = list(map(set, items))
rules_df_35['Antecedent'] = list(map(set, antecedent))
rules_df_35['Consequent'] = list(map(set, consequent))
rules_df_35['Support'] = support
rules_df_35['Confidence'] = confidence
rules_df_35['Lift'] = lift

# sorting the dataframe by lift
rules_df_35.sort_values(by=['Lift', 'Support', 'Confidence'], ascending = [False, False, False], inplace = True)

In [27]:
rules_df_35.head(20)

,Items,Antecedent,Consequent,Support,Confidence,Lift
1171,{Married/same-sex civil partnership couple fam...,{Married or in a registered same-sex civil par...,{Married/same-sex civil partnership couple fam...,0.356224,0.953525,2.278457
1167,{Married/same-sex civil partnership couple fam...,{Married/same-sex civil partnership couple fam...,{Married or in a registered same-sex civil par...,0.356224,0.851200,2.278457
713,{Married or in a registered same-sex civil par...,{Married/same-sex civil partnership couple fam...,{Married or in a registered same-sex civil par...,0.356224,0.851200,2.274353
1177,{Married/same-sex civil partnership couple fam...,{Married/same-sex civil partnership couple fam...,{Married or in a registered same-sex civil par...,0.356224,0.851200,2.274353
709,{Married or in a registered same-sex civil par...,{Married or in a registered same-sex civil par...,{Married/same-sex civil partnership couple fam...,0.356224,0.951808,2.274353
1162,{Married/same-sex civil partnership couple fam...,{Married or in a registered same-sex civil par...,"{No, Married/same-sex civil partnership couple...",0.356224,0.951808,2.274353
913,{Married or in a registered same-sex civil par...,{Married or in a registered same-sex civil par...,{Married/same-sex civil partnership couple fam...,0.356224,0.949382,2.268555
1166,{Married/same-sex civil partnership couple fam...,{Married or in a registered same-sex civil par...,{Married/same-sex civil partnership couple fam...,0.356224,0.949382,2.268555
914,{Married or in a registered same-sex civil par...,{Married/same-sex civil partnership couple fam...,{Married or in a registered same-sex civil par...,0.356224,0.851200,2.268555
1172,{Married/same-sex civil partnership couple fam...,{Married/same-sex civil partnership couple fam...,{Married or in a registered same-sex civil par...,0.356224,0.851200,2.268555


In [28]:
result = rules_df_35[rules_df_35['Lift'] > 1]

In [29]:
#writing to a file
filepath = Path("/Users/sudeshgurung/Msc-Data_Analytics/ADTA_with_python/Course_work_1/rules_ARM/rules_35sup_50con.csv")
filepath.parent.mkdir(parents=True, exist_ok=True)  
result.to_csv(filepath) 